In [1]:
# Just regression
# import
import csv
import math
import random
import sys
import numpy as np
import matplotlib.pyplot as plt
from pystruct.models import ChainCRF
from pystruct.learners import FrankWolfeSSVM

%matplotlib inline

random.seed

<bound method Random.seed of <random.Random object at 0x10283bc18>>

In [2]:
data_num = 0
available_wifi = 0
building = 1
floor = 2
with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if (row[523] == 'BUILDINGID'):
            continue
        elif (int(row[523]) is not building or int(row[522]) is not floor):
            continue
        data_num += 1

print(data_num)
i = -1

wifi_loc_time = np.zeros(shape = (data_num, 524))
wifi = np.zeros(shape = (data_num, 520))
longitude = np.zeros(shape=(data_num,2))

with open("1478167720_9233432_trainingData.csv", newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        if (row[523] == 'BUILDINGID'):
            continue
        elif (int(row[523]) is not building or int(row[522]) is not floor):
            continue
        i = i+1
        if (i > data_num):
            break
        # wifi
        wifi_loc_time[i-1][:520] = np.array(row[:520])
        wifi[i-1][:] = np.array(row[:520])
        # location x, y
        wifi_loc_time[i-1][520:522] = np.array(row[520:522])
        longitude[i-1][:] = np.array(row[520:522])
        # userID
        wifi_loc_time[i-1][522] = np.array(row[526])
        # time stamp
        wifi_loc_time[i-1][-1] = np.array(row[-1])

1396


In [3]:
print(wifi_loc_time[:,-1])

[1.37171369e+09 1.37171410e+09 1.37171381e+09 ... 1.37171510e+09
 1.37171506e+09 1.37171373e+09]


In [4]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
                                              ExpSineSquared, DotProduct,
                                              ConstantKernel, WhiteKernel)

In [5]:
# Testing
# extract data
data_num_v = 0 # 86 in validation dataset
with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
    spamreader_v = csv.reader(v_csvfile, delimiter=',')
    for row in spamreader_v:
        if (row[523] == 'BUILDINGID'):
            continue
        elif (int(row[523]) is not building or int(row[522]) is not floor):
            continue
        data_num_v += 1

print(data_num_v)
i = -1

wifi_loc_time_v = np.zeros(shape = (data_num_v, 524))
wifi_v = np.zeros(shape = (data_num_v, 520))
longitude_v = np.zeros(shape=(data_num_v,2))

# 520 WAP
# FLOOR 2
# Building 1
# timestamp

# total data num = 19938

with open("1478167721_0345678_validationData.csv", newline='') as v_csvfile:
    spamreader_v = csv.reader(v_csvfile, delimiter=',')
    for row in spamreader_v:
        if (row[523] == 'BUILDINGID'):
            continue
        elif (int(row[523]) is not building or int(row[522]) is not floor):
            continue
        i = i+1
        #if (i > data_num):
        #    break
        # wifi
        wifi_loc_time_v[i-1][:520] = np.array(row[:520])
        wifi_v[i-1][:] = np.array(row[:520])
        # location x, y
        wifi_loc_time_v[i-1][520:522] = np.array(row[520:522])
        longitude_v[i-1][:] = np.array(row[520:522])
        # userID
        wifi_loc_time_v[i-1][522] = np.array(row[526])
        # time stamp
        wifi_loc_time_v[i-1][-1] = np.array(row[-1])

87


In [ ]:
len_scale = 1.0
ap = 0.1
for i in range(999):
    len_scale -= 0.001
    for ap in range(10000):
        ap += 10.0
        kernel = 1.0 * RationalQuadratic(length_scale=len_scale, alpha=ap)
        gp = GaussianProcessRegressor(kernel=kernel,
                                          alpha=0.5).fit(wifi, longitude)
        loc_pred, loc_cov = gp.predict(wifi_v, return_cov=True)
        RMSE = 0
        for i in range(len(loc_pred)):
                #print(loc_pred[i,0], longitude_v[i,0], loc_pred[i,0]-longitude_v[i,0])
                #print(loc_pred[i,1], longitude_v[i,1], loc_pred[i,1]-longitude_v[i,1])
                #print(math.sqrt(pow(loc_pred[i,0]-longitude_v[i,0],2)+pow(loc_pred[i,1]-longitude_v[i,1],2)))
            RMSE+=math.sqrt(pow(loc_pred[i,0]-longitude_v[i,0],2)+pow(loc_pred[i,1]-longitude_v[i,1],2))
        print(len_scale, ap, RMSE/data_num_v)

0.999 10.0 15.297912449643768
0.999 11.0 15.29790962063512
0.999 12.0 15.297906890238991
0.999 13.0 15.297904296468836
0.999 14.0 15.297901867965155
0.999 15.0 15.29789959653876
0.999 16.0 15.297897481191445
0.999 17.0 15.297895511257726
0.999 18.0 15.297893674253235
0.999 19.0 15.297891953739645
0.999 20.0 15.297890348271734
0.999 21.0 15.297888838486935
0.999 22.0 15.297887422076426
0.999 23.0 15.297886079375354
0.999 24.0 15.298116903024027
0.999 25.0 15.298116861906763
0.999 26.0 15.298116822085458
0.999 27.0 15.298116784289368
0.999 28.0 15.298116747153296
0.999 29.0 15.298116712418645
0.999 30.0 15.29811667860911
0.999 31.0 15.29811664480548
0.999 32.0 15.298116613712903
0.999 33.0 15.298116582148928
0.999 34.0 15.298116553770447
0.999 35.0 15.298116526236958
0.999 36.0 15.298116498327607
0.999 37.0 15.298116471979748
0.999 38.0 15.298116447545025
0.999 39.0 15.29811642341678
0.999 40.0 15.298116400148261
0.999 41.0 15.298116377522609
0.999 42.0 15.298116356301474
0.999 43.0 15.2

In [ ]:
# from sklearn.decomposition import PCA, KernelPCA

In [ ]:
# n_components = 2
# # pca = PCA(n_components=n_components)
# # wifi_pca = pca.fit_transform(wifi)
# kpca = KernelPCA(kernel="rbf", n_components=2, gamma=100)
# wifi_pca = kpca.fit_transform(wifi)

In [ ]:
# for i in range(len(wifi)):
#     axes = plt.gca()
#     axes.set_xlim([min(wifi_pca[:,0]),max(wifi_pca[:,0])])
#     axes.set_ylim([min(wifi_pca[:,1]),max(wifi_pca[:,1])])
#     plt.scatter(wifi_pca[:,0], wifi_pca[:,1], c='r')
#     plt.annotate(i, (wifi_pca[i,0], wifi_pca[i,1]))
#     plt.show()

In [ ]:
plt.plot(longitude[:,0], longitude[:,1], 'b.')

In [ ]:
# print(wifi_pca)

In [ ]:
# print(longitude)

In [ ]:
# wifi_pca_v = kpca.fit_transform(wifi_v)
# for kernel in kernels:
#     gp = GaussianProcessRegressor(kernel=kernel,
#                                       alpha=0.5).fit(wifi_pca, longitude)
#     loc_pred, loc_cov = gp.predict(wifi_pca_v, return_cov=True)
#     RMSE = 0
#     for i in range(len(loc_pred)):
#             #print(loc_pred[i,0], longitude_v[i,0], loc_pred[i,0]-longitude_v[i,0])
#             #print(loc_pred[i,1], longitude_v[i,1], loc_pred[i,1]-longitude_v[i,1])
#             #print(math.sqrt(pow(loc_pred[i,0]-longitude_v[i,0],2)+pow(loc_pred[i,1]-longitude_v[i,1],2)))
#         RMSE+=math.sqrt(pow(loc_pred[i,0]-longitude_v[i,0],2)+pow(loc_pred[i,1]-longitude_v[i,1],2))
#     print(RMSE/data_num_v)

In [ ]:
print(1e+05)